### Question 1: Overall spending effect**
- Compare total spending per cycle by vehicle type: direct contributions, independent expenditures, 527 spending
- Fit interrupted time-series model with 2010 as intervention point
- **Primary metric:** Ratio of independent expenditures to total spending per cycle
- **Key differentiator (from proposal):** Analyze whether the "tilted playing field" is driven by traditional corporate PAC structures (which only saw ~14% increase) or the newer Super PAC/IE vehicles


In [2]:
# Load packages + connected to database

import sqlite3
import pandas as pd
import matplotlib.pyplot as plt


conn = sqlite3.connect('../data/citizens_united.db')


## 1. Load and aggregate spending by type

Query `partisan_spending_monthly` — the core derived table built by `02_clean.py`. Sum all spending by cycle and vehicle type (individual donations, direct PAC contributions, independent expenditures, 527 spending), inflation-adjusted to 2024 dollars. Exclude `unaligned` rows to keep only clearly partisan spending.

In [3]:
query = """
     SELECT Cycle, era, spending_type,
            SUM(total_amount_2024) AS total,
            SUM(n_transactions)    AS n_trans
     FROM partisan_spending_monthly
     WHERE partisan_direction != 'unaligned'
     GROUP BY Cycle, era, spending_type
     ORDER BY Cycle, spending_type
   """

df = pd.read_sql(query, conn)

pivot = df.pivot(index='Cycle',
            columns='spending_type',
            values='total').fillna(0)

pivot


spending_type,527,individual,pac_direct,pac_independent
Cycle,,,,
2004,7.726287e+08,6.947881e+08,5.582795e+06,4.022877e+08
2008,3.114690e+08,1.042066e+09,4.701335e+06,3.299818e+08
2012,1.792782e+08,1.001564e+09,2.763514e+06,8.906821e+08
2020,5.687093e+08,9.583325e+08,1.828808e+06,1.386246e+09


## 2. Compute derived metrics

Add columns to `pivot`:
- `outside_spending` = `pac_independent` + `527` — the full pre/post-CU outside spending picture (527s were the pre-CU vehicle; IEs replaced them post-CU)
- `total` = sum of all four vehicle types per cycle
- `ie_ratio` = share of total spending that is independent expenditures (the primary CU metric)
- `outside_ratio` = share that is any form of outside spending (IE + 527)

In [4]:
pivot['outside_spending'] = pivot['pac_independent'] + pivot['527']
pivot['total']            = pivot.sum(axis=1)
pivot['ie_ratio']         = pivot['pac_independent'] / pivot['total']
pivot['outside_ratio']    = pivot['outside_spending'] / pivot['total']

pivot

spending_type,527,individual,pac_direct,pac_independent,outside_spending,total,ie_ratio,outside_ratio
Cycle,,,,,,,,
2004,7.726287e+08,6.947881e+08,5.582795e+06,4.022877e+08,1.174916e+09,3.050204e+09,0.131889,0.385193
2008,3.114690e+08,1.042066e+09,4.701335e+06,3.299818e+08,6.414508e+08,2.329669e+09,0.141643,0.275340
2012,1.792782e+08,1.001564e+09,2.763514e+06,8.906821e+08,1.069960e+09,3.144249e+09,0.283273,0.340291
2020,5.687093e+08,9.583325e+08,1.828808e+06,1.386246e+09,1.954956e+09,4.870073e+09,0.284646,0.401422


## 3. Key metrics — IE vs. Direct PAC growth (2008 → 2012)

Compute the headline numbers from the literature:
- PAC independent expenditures grew ~594% from 2008 to 2012 (the CU effect)
- PAC direct contributions grew only ~14% over the same period (unaffected by CU, subject to same limits)

The contrast between these two growth rates is the core empirical argument of the paper.

In [5]:
ie_2008 = pivot.loc['2008', 'pac_independent']
ie_2012 = pivot.loc['2012', 'pac_independent']
ie_growth_pct = (ie_2012 - ie_2008) / ie_2008 * 100   

direct_2008 = pivot.loc['2008', 'pac_direct']
direct_2012 = pivot.loc['2012', 'pac_direct']
direct_growth_pct = (direct_2012 - direct_2008) / direct_2008 * 100

## 4. Print summary

Print the key growth figures and the full pivot table in readable form.

## 5. Figure 1 — Total spending by type (stacked bar)

One bar per cycle, stacked by vehicle type (bottom to top: individual → pac_direct → pac_independent → 527). A vertical dashed line between the 2008 and 2012 bars marks Citizens United (Jan 21, 2010). This is the primary Q1 figure showing both the growth in total spending and the shift in its composition.

Y-axis in 2024 $B. Include OpenSecrets attribution.

## 6. Figure 2 — IE share of total spending over time

Line chart with one point per cycle showing `ie_ratio` (pac_independent / total) as a percentage. The jump between 2008 and 2012 is the visual signature of Citizens United. Add the CU vertical line and label.

## 7. Figure 3 — Direct vs. Independent PAC spending (2008 → 2012)

Grouped bar chart with two groups (pac_direct, pac_independent), each showing the 2008 and 2012 values side by side. Annotate the bars with the growth percentages computed above. This is the "~14% vs. ~594%" comparison that is the key differentiator from prior work.

## 8. Save summary table

Save `pivot` (with all derived columns) to `output/tables/q1_spending_by_type.csv`. Create the output directories if they don't exist.